Collecting data from different sources for a set of addresses (we can provide you with sample addresses but feel free to find interesting groups of wallets
which are publicly available); 

the data to be collected by the list of wallets will
include transactions, tokentransfers, balance/token balance

In [1]:
## import libraries
import numpy as np
import os
import pandas as pd
import requests
import sys
import time

In [2]:
## import configuration
sys.path.append('..')
from config.addresses import addresses

In [3]:
## etherscan API
etherscan_api_url = 'https://api.etherscan.io/api'
with open('../config/etherscan_api.tkn') as tkn_file:
    etherscan_token = tkn_file.read()
    
balance_request = '?module=account&action=balance&address={address}&tag=latest&apikey={etherscan_token}'
transaction_request = '?module=account&action=txlist&address={address}&startblock=0&endblock=99999999' \
    '&page=1&offset=10000&sort=desc&apikey={etherscan_token}'

In [4]:
## accounts dataframe
data = pd.DataFrame(columns=['Name', 'Address', 'CurrentBalanceEth'])

# current balance
for name, address in addresses.items():
    current_balance = requests.get(
        etherscan_api_url + balance_request.format(address=address, etherscan_token=etherscan_token)) \
        .json()['result']
    data = data.append({'Name': name,
                        'Address': address,
                        'CurrentBalanceWei': int(current_balance),
                       }, ignore_index=True)

In [5]:
## get all transactions (API limits to 10,000 results)
transactions_df = pd.DataFrame()
start_time = time.time()
for name, address in addresses.items():
    transactions = requests.get(
        etherscan_api_url \
        + transaction_request.format(address=address, etherscan_token=etherscan_token)) \
        .json()['result']
    for transaction in transactions:
        transactions_df = transactions_df.append({
            'from': transaction['from'],
            'to': transaction['to'],
            'timeStamp': transaction['timeStamp'],
            'value': transaction['value'],
            'contractAddress': transaction['contractAddress']
            }, ignore_index=True)
    print(address)
end_time = time.time()

0x05a56e2d52c817161883f50c441c3228cfe54d9f
0x514910771af9ca656af840dff83e8264ecf986ca
0x1f9840a85d5af5bf1d1762f925bdaddc4201f984
0xeB2629a2734e272Bcc07BDA959863f316F4bD4Cf
0x53d284357ec70cE289D6D64134DfAc8E511c8a3D
0x00000000219ab540356cbb839cbe05303d7705fa
0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2
0x0c54FcCd2e384b4BB6f2E405Bf5Cbc15a017AaFb
0xde0B295669a9FD93d5F28D9Ec85E40f4cb697BAe
0x7813C776cD8EAb537028b4499d467b1f1B86b14C
0xedDF0Df82006a477C87C2E16FfdEF0d97631E941
0x9f966149B7Dd6AB61440Ec4D4B853f4605739E73
0x630163B84674B2B404fB6036A510574F259c5Cb7
0x724D08F4688Cda05d8e3243db9Db1B20C90f3a05
0x5C3d1309D8b6e37EFF9FD6C258e1544549b39D22
0x4bE991B4d560BBa8308110Ed1E0D7F8dA60ACf6A
0x6a1Ef9bF93048533c49a1Eed984c080608f7DB6A
0xdC182F6E7461Fe1fE442665615aA44F70f978Db3
0xe4E345594375B54F395B54C41bD8b370A302bf69
0x06812A2035BDa4707107539725902e065622CEE7
0x8093150EC164753994A1F65616E04Ae92a9Ef8c5
0x29236dFcae0aEE2D6da157F3B6835830c75875Ad
0x33CCa8E7420114dB103d61bd39A72ff65E46352D
0x7009033C0

In [6]:
address

'0x989B836D68700DA948B5c04A65b3bBA39F400ad7'

In [7]:
print("Time to obtain last 10,000 transactions for each account: {:0.2f} minutes".format( (end_time-start_time)/60 ))

Time to obtain last 10,000 transactions for each account: 9.79 minutes


In [8]:
## join balance with transaction dataset on from and to of account address
transactions_df

,from,to,timeStamp,value,contractAddress
0,0x4c549990a7ef3fea8784406c1eecc98bf4211fa5,0x05a56e2d52c817161883f50c441c3228cfe54d9f,1605819070,1229528978404428,
1,0x3b4a51b7ce963f67eff60d8b9a7748fea93bd9a4,0x05a56e2d52c817161883f50c441c3228cfe54d9f,1555768341,100000000000000,
2,0x3b4a51b7ce963f67eff60d8b9a7748fea93bd9a4,0x05a56e2d52c817161883f50c441c3228cfe54d9f,1555767462,100000000000000,
3,0x0975629a80cf23d706358093891b655e1267bb17,0x05a56e2d52c817161883f50c441c3228cfe54d9f,1547295564,20000000000000000,
4,0xb5fb6e595568b53115c4fea8f3ef005d908bf17d,0x05a56e2d52c817161883f50c441c3228cfe54d9f,1547295205,1000000000000000,
...,...,...,...,...,...
101060,0x501698a6f6f762c79e4d28e3815c135e3f9af996,0x989b836d68700da948b5c04a65b3bba39f400ad7,1612992337,0,
101061,0x0699a397c3cf614c9a7db23a4be28fc4c8f3a755,0x989b836d68700da948b5c04a65b3bba39f400ad7,1612988554,0,
101062,0x536ee6348b3790ec74c21fcbe3d990066e9b8ec7,0x989b836d68700da948b5c04a65b3bba39f400ad7,1612976921,0,
101063,0x536ee6348b3790ec74c21fcbe3d990066e9b8ec7,0x989b836d68700da948b5c04a65b3bba39f400ad7,1612976781,0,


In [9]:
##transactions[0]

In [10]:
transactions[1]

{'blockNumber': '13093580',
 'timeStamp': '1629881375',
 'hash': '0xc72dff267877228c3a7d340b15e17396128f58e1f7038246fe45065057ba1a2d',
 'nonce': '255579',
 'blockHash': '0xb2ff69ab63520af9159bebd7b0130363063449ff038bc2c810814e3dc54a8a54',
 'transactionIndex': '104',
 'from': '0xc7310123914f624da9c376f8ec590055e62733c1',
 'to': '0x989b836d68700da948b5c04a65b3bba39f400ad7',
 'value': '0',
 'gas': '500000',
 'gasPrice': '83000000000',
 'isError': '0',
 'txreceipt_status': '1',
 'input': '0x202ee0ed000000000000000000000000000000000000000000000000000000000000029300000000000000000000000000000000000000000000000001b47f638f78d84a',
 'contractAddress': '',
 'cumulativeGasUsed': '4792088',
 'gasUsed': '111204',
 'confirmations': '8750'}

In [11]:
address

'0x989B836D68700DA948B5c04A65b3bBA39F400ad7'